þarf að laga events function þannig að það geti skilað kk og kvk events. Og skoða öll hin functions þ.a. þau séu þannig flex að ég get sett inn kk eða kvk eða bæði og fengið það sem ég vill

commenta hvað hvaða function er að gera

In [323]:
# for the data
import pandas as pd
import numpy as np
import requests
import re

In [324]:
def games_json_dump(year=2022, division=1):
    url = f"https://c3po.crossfit.com/api/leaderboards/v2/competitions/games/{year}/leaderboards?division={division}&sort=0&page=1"
    #Sends the request and reads the response.
    response = requests.get(url)
    #Loads response as JSON
    response = response.json()
    return(response)

In [338]:
def games_info(year=2022, division=1):
    # get the json data from the web service
    response = games_json_dump(year=year, division=division)
    # collecting data on the specified games session
    total_pages = response['pagination']['totalPages']
    total_competitors = response['pagination']['totalCompetitors']
    competition_id = response['competition']['competitionId']
    total_events = len(response['ordinals'])
    # return a list of responses
    games_info_list = [competition_id, total_pages, total_competitors, total_events]
    return(games_info_list)

In [340]:
# held að ég sé ekkert að nota þetta en mögulega gott að nota þetta til að byggja upp gagnamódelið
def games_info_multiple(year_from=2018, year_too=2022, division=1):
    # create a dataframe to store results
    df_games_info_multiple = pd.DataFrame(columns=['competitionId', 'totalPages', 'totalCompetitors', 'totalEvents', 'year','division'])
    # determining what division value we need
    if division.lower() == "male":
        division_value = [1]
    elif division.lower() == "female":
        division_value = [2]
    # else we just take both divisions
    else:
        division_value = [1,2]
    # iterate over yearFrom and yearToo
    for i in range(0, year_too - year_from + 1):
        for j in range(0,len(division_value)):
            # results form running gamesInfo funcation
            result_list = games_info(year_from+i,division=division_value[j])
            # results from function
            result_list.append(year_from+i)
            result_list.append(division_value[j])
            # Convert the list to a DataFrame with a single row
            temp_df = pd.DataFrame([result_list], columns=['competitionId', 'totalPages', 'totalCompetitors', 'totalEvents', 'year','division'])
            # Concatenate the DataFrame with the existing DataFrame
            df_games_info_multiple = pd.concat([df_games_info_multiple, temp_df], ignore_index=True)
    return(df_games_info_multiple)

In [341]:
df_games_info_multiple = games_info_multiple(2021,2022,"")
df_games_info_multiple.head()

,competitionId,totalPages,totalCompetitors,totalEvents,year,division
0,161,1,40,15,2021,1
1,161,1,40,15,2021,2
2,189,1,40,14,2022,1
3,189,1,40,14,2022,2


In [345]:
def games_info_competitors(year, division):
    url = f"https://c3po.crossfit.com/api/leaderboards/v2/competitions/games/{year}/leaderboards?division={division}&sort=0"
    #Sends the request and reads the response.
    response = requests.get(url)
    #Loads response as JSON
    response = response.json()
    # create a dataframe to store info
    df_games_info_competitors = pd.DataFrame()
    # collecting data on the specified games session
    info = games_info(year=year,division=division)
    for i in range(0, info[2]):
        entrant_json = response['leaderboardRows'][i]['entrant']
        df_temp = pd.DataFrame([entrant_json])
        # Concatenate the DataFrame with the existing DataFrame
        df_games_info_competitors = pd.concat([df_games_info_competitors, df_temp], ignore_index=True)
    return(df_games_info_competitors)

In [346]:
df_games_info_competitors = games_info_competitors(2022,1)

In [347]:
df_games_info_competitors.head(5)

,competitorId,competitorName,firstName,lastName,status,postCompStatus,gender,profilePicS3key,countryOfOriginCode,countryOfOriginName,countryShortCode,regionId,regionName,divisionId,affiliateId,affiliateName,age,height,weight,teamCaptain
0,811708,Justin Medeiros,Justin,Medeiros,ACT,,M,672d4-P811708_4-184.jpg,US,United States,,31,North America,1,1792,CrossFit Fort Vancouver,23,69 in,195 lb,0
1,975774,Roman Khrennikov,Roman,Khrennikov,ACT,,M,a2263-P975774_2-184.jpg,RU,,,28,Asia,1,None,,27,182 cm,103 kg,0
2,40955,Ricky Garard,Ricky,Garard,ACT,,M,89cbe-P40955_10-184.jpg,AU,Australia,,32,Oceania,1,None,,28,179 cm,200 lb,0
3,74320,Samuel Kwant,Samuel,Kwant,ACT,,M,a9afa-P74320_4-184.jpg,US,United States,,31,North America,1,None,,26,71 in,206 lb,0
4,469656,Jeffrey Adler,Jeffrey,Adler,ACT,,M,e480e-P469656_1-184.jpg,CA,Canada,,31,North America,1,18059,CrossFit Wonderland,28,69 in,197 lb,0


In [351]:
# Two functions to make height and weight comparable between US and rest of world

# function to convert inches to cm
def inch_to_cm(height):
    if 'in' in height:
        height_in_inches = float(height.replace('in', '').strip())
        return round(height_in_inches * 2.54, 1)
    else:
        return float(height.replace('cm', '').strip())

# function to convert lbs to kg
def lb_to_kg(weight):
    if 'lb' in weight:
        weight_in_lbs = float(weight.replace('lb', '').strip())
        return round(weight_in_lbs / 2.20462, 1)
    else:
        return float(weight.replace('kg', '').strip())

In [352]:
# apply the function to the 'Height' column and assign it back to the column
competitorsInfo['heightInCm'] = competitorsInfo['height'].apply(inch_to_cm)
competitorsInfo['weightInKg'] = competitorsInfo['weight'].apply(lb_to_kg)

# print the updated dataframe
competitorsInfo.head(5)

,competitorId,competitorName,firstName,lastName,status,postCompStatus,gender,profilePicS3key,countryOfOriginCode,countryOfOriginName,...,regionName,divisionId,affiliateId,affiliateName,age,height,weight,teamCaptain,heightInCm,weightInKg
0,811708,Justin Medeiros,Justin,Medeiros,ACT,,M,672d4-P811708_4-184.jpg,US,United States,...,North America,1,1792,CrossFit Fort Vancouver,23,69 in,195 lb,0,175.3,88.5
1,975774,Roman Khrennikov,Roman,Khrennikov,ACT,,M,a2263-P975774_2-184.jpg,RU,,...,Asia,1,None,,27,182 cm,103 kg,0,182.0,103.0
2,40955,Ricky Garard,Ricky,Garard,ACT,,M,89cbe-P40955_10-184.jpg,AU,Australia,...,Oceania,1,None,,28,179 cm,200 lb,0,179.0,90.7
3,74320,Samuel Kwant,Samuel,Kwant,ACT,,M,a9afa-P74320_4-184.jpg,US,United States,...,North America,1,None,,26,71 in,206 lb,0,180.3,93.4
4,469656,Jeffrey Adler,Jeffrey,Adler,ACT,,M,e480e-P469656_1-184.jpg,CA,Canada,...,North America,1,18059,CrossFit Wonderland,28,69 in,197 lb,0,175.3,89.4


In [321]:
# converting the height string to total inches so I can calculate height in cm later
heightInFeet = int(competitorData['entrant']['height'][:1])
# to account for if the inches are 10 or over
if "cm" in competitorData['entrant']['height']:
    heightInInches = int(competitorData['entrant']['height'][2:4])
else:
    heightInInches = int(competitorData['entrant']['height'][2:3])
df_cfgMales.loc[i,'heightInInches'] = heightInFeet*12 + heightInInches

In [291]:
def gamesInfoScores(year=2022, division=1):
    # create a dataframe to store the scores
    scoresDf = pd.DataFrame()
    # collecting data on the specified games session
    gamesInfoCall = gamesInfo(year=year,division=division)
    # determining what division value we need
    if division == 1:
        divisionValue = [1]
    elif division == 2:
        divisionValue = [2]
    # else we just take both divisions
    else:
        divisionValue = [1,2]
    
    for j in range(0,len(divisionValue)):
        # get the json data from the web service
        response = gamesJsonDump(year=year, division=divisionValue[j])
        for i in range(0, gamesInfoCall[2]):
            # results form running gamesInfo funcation
            scoresJson = response['leaderboardRows'][i]['scores']
            tempDf = pd.json_normalize(scoresJson)
            tempDf['competitorId'] = response['leaderboardRows'][i]['entrant']['competitorId']
            tempDf['division'] = divisionValue[j]
            # Concatenate the DataFrame with the existing DataFrame
            scoresDf = pd.concat([scoresDf, tempDf], ignore_index=True)
    # transformation of the data
    # some athletes got a DF score due to withdrawing from competiton before first event
    # if len(competitorData['overallScore']) < 1:
    #     df_cfgMales.loc[i,'overallScore'] = "0" 

    return(scoresDf)

SyntaxError: non-default argument follows default argument (581231218.py, line 1)

In [294]:
gamesInfoScores(2020,1)

,ordinal,rank,score,valid,scoreDisplay,scoreIdentifier,mobileScoreDisplay,scaled,video,heat,lane,breakdown,competitorId,division
0,1,1,100,1,3:08,fd69cb83976042a40b2f,,0,0,,,100 pts,153604,1
1,2,4,88,1,425 lb,c19135e3cd0bc7579764,,0,0,,,88 pts,153604,1
2,3,1,100,1,2:44,330d758bbd555ba25ecd,,0,0,,,100 pts,153604,1
3,4,8,72,1,2:55.20,388301719cfceaba4bc7,,0,0,,,72 pts,153604,1
4,5,1,100,1,17:50,48118b0d264137489e00,,0,0,,,100 pts,153604,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,16,,0,,,5e28fd0c71aeed055437,,0,0,,,,234569,1
596,17,,0,,,9b52a55b3e7c1f0a4985,,0,0,,,,234569,1
597,18,,0,,,e36ac94cf7efa29af66d,,0,0,,,,234569,1
598,19,,0,,,37a19dfa21ea4ff4c02c,,0,0,,,,234569,1


In [263]:
gamesInfo(2019,1)

[52, 3, 144, 12]

In [253]:
results = pd.merge(gamesInfoScoresFinal, competitorsInfo[['competitorId','competitorName']], on='competitorId')

In [255]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 559
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ordinal             560 non-null    int64 
 1   rank                560 non-null    object
 2   score               560 non-null    object
 3   valid               560 non-null    object
 4   scoreDisplay        560 non-null    object
 5   scoreIdentifier     560 non-null    object
 6   mobileScoreDisplay  560 non-null    object
 7   scaled              560 non-null    object
 8   video               560 non-null    object
 9   heat                560 non-null    object
 10  lane                560 non-null    object
 11  breakdown           521 non-null    object
 12  competitorId        560 non-null    object
 13  competitorName      560 non-null    object
dtypes: int64(1), object(13)
memory usage: 65.6+ KB


In [259]:
# convert to numeric and check for NaN values
non_numeric = df[pd.to_numeric(df['scoreDisplay'], errors='coerce').isnull()]
non_numeric

,ordinal,rank,score,valid,scoreDisplay,scoreIdentifier,mobileScoreDisplay,scaled,video,heat,lane,breakdown
0,1,1,100,1,36:47.61,8cd42e7335a9f58c2c5c,,0,0,,,100 pts
1,2,2,97,1,06:58.74,47406f45a1013a3d1af7,,0,0,,,97 pts
2,3,3,94,1,16 rep,fc8ecef95523ddaa4fcb,,0,0,,,94 pts
3,4,5,88,1,5 pt,2991f049897ee92f2fd9,,0,0,,,88 pts
4,5,7,82,1,09:49.82,54c34a0f6157e81313d0,,0,0,,,82 pts
...,...,...,...,...,...,...,...,...,...,...,...,...
9,10,33,16,1,04:49.82,c9faee5905ac44a713a8,,0,0,,,16 pts
10,11,20,43,1,310 lb,4be1ee3a0ce29eaca7c3,,0,0,,,43 pts
11,12,31,0,,,2e71160ae2cfcffa9fcb,,0,0,,,NaN
12,13,30,0,,,ade47f11f203f75e4d19,,0,0,,,NaN


In [7]:
# create a dataframe that contains one line for every male competitor
df_cfgMales = pd.DataFrame(np.zeros((totalMaleCompetitors,11)),columns=['competitorId','competitorName','overallRank', 'overallScore',
                                                                       'height', 'heightInInches', 'weight', 'weightInKg',
                                                                        'age',
                                                                       'countryOfOriginName','affiliateName'])
# just filling the dataframe with nan data
df_cfgMales = df_cfgMales.replace(0,np.nan)

# loop over all events for all competitors
for i in range(0,totalMaleCompetitors-1):
    competitorData = responseMales['leaderboardRows'][i]
        
    # competition data
    df_cfgMales.loc[i,'overallRank'] = competitorData['overallRank']
    # some athletes got a DF score due to withdrawing from competiton before first event
    if len(competitorData['overallScore']) < 1:
        df_cfgMales.loc[i,'overallScore'] = "0"
    else:
        df_cfgMales.loc[i,'overallScore'] = competitorData['overallScore']      
    # get some personal data!
    df_cfgMales.loc[i,'competitorId'] = str(competitorData['entrant']['competitorId'])
    df_cfgMales.loc[i,'competitorName'] = competitorData['entrant']['competitorName']
    df_cfgMales.loc[i,'countryOfOriginName'] = competitorData['entrant']['countryOfOriginName']
    df_cfgMales.loc[i,'height'] = competitorData['entrant']['height']
    
    # converting the height string to total inches so I can calculate height in cm later
    heightInFeet = int(competitorData['entrant']['height'][:1])
    # to account for if the inches are 10 or over
    if len(competitorData['entrant']['height']) > 4:
        heightInInches = int(competitorData['entrant']['height'][2:4])
    else:
        heightInInches = int(competitorData['entrant']['height'][2:3])
    df_cfgMales.loc[i,'heightInInches'] = heightInFeet*12 + heightInInches
    
    df_cfgMales.loc[i,'age'] = competitorData['entrant']['age']
    # clean lbs from weight
    df_cfgMales.loc[i,'weight'] = re.sub("lbs", "", competitorData['entrant']['weight'])
    try:
        df_cfgMales.loc[i,'affiliateName'] = competitorData['entrant']['affiliateName']
    except:
        df_cfgMales.loc[i,'affiliateName'] = 'Unaffiliated'
    # clean the "T" from workoutRank, this is when athletes are tied in a workout
    try:
        df_cfgMales.loc[i,'overallRank'] = re.sub("\D", "", df_cfgMales.loc[i,'overallRank'])
    except:
        df_cfgMales.loc[i,'overallRank'] = df_cfgMales['overallRank']
    df_cfgMales.loc[i,'overallRank'] = int(df_cfgMales.loc[i,'overallRank'])
# change overallRank and overallScore to a numeric value so we can to some calculations
df_cfgMales['overallRank'] = df_cfgMales['overallRank'].astype('int')
df_cfgMales['overallScore'] = df_cfgMales['overallScore'].astype('int')
df_cfgMales['age'] = df_cfgMales['age'].astype('int')
df_cfgMales['weight'] = df_cfgMales['weight'].astype('int')
# create a new columns and convert height to cm and wieght to kg
df_cfgMales['heightInCm'] = round(df_cfgMales['heightInInches'].apply(lambda x: x*2.54),1)
df_cfgMales['weightInKg'] = round(df_cfgMales['weight'].apply(lambda x: x/2.205),1)

ValueError: invalid literal for int() with base 10: ''

In [26]:
df_cfgMales

,competitorId,competitorName,overallRank,overallScore,height,heightInInches,weight,weightInKg,age,countryOfOriginName,affiliateName,heightInCm
0,153604,Mathew Fraser,1,984,"5'7""",67.0,195,88.4,29,United States,CrossFit Mayhem,170.2
1,2725,Noah Ohlsen,2,949,"5'7""",67.0,190,86.2,28,United States,Peak 360 CrossFit,170.2
2,81616,Björgvin Karl Guðmundsson,3,888,"5'10""",70.0,185,83.9,26,Iceland,CrossFit Hengill,177.8
3,34796,Scott Panchik,4,795,"5'9""",69.0,187,84.8,31,United States,CrossFit Mentality,175.3
4,16080,James Newbury,5,728,"5'10""",70.0,192,87.1,28,Australia,CrossFit Mode,177.8
5,199938,Jacob Heppner,6,694,"5'8""",68.0,192,87.1,29,United States,Cobra Command CrossFit,172.7
6,660204,Matt Mcleod,7,671,"5'6""",66.0,181,82.1,26,Australia,CrossFit Crossaxed,167.6
7,698182,Adrian Mundwiler,8,632,"5'8""",68.0,187,84.8,26,Switzerland,Unaffiliated,172.7
8,310970,Saxon Panchik,9,587,"5'9""",69.0,185,83.9,23,United States,CrossFit Cliffside,175.3
9,35627,Will Moorad,10,485,"5'9""",69.0,190,86.2,30,United States,CrossFit Trivium,175.3
